<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Spark-MLlib-Tuning" data-toc-modified-id="Spark-MLlib-Tuning-1"><span class="toc-item-num">1&nbsp;&nbsp;</span><a href="https://spark.apache.org/docs/latest/ml-tuning.html" target="_blank">Spark MLlib Tuning</a></a></span></li><li><span><a href="#Hyperopt" data-toc-modified-id="Hyperopt-2"><span class="toc-item-num">2&nbsp;&nbsp;</span><a href="https://github.com/hyperopt/hyperopt" target="_blank">Hyperopt</a></a></span><ul class="toc-item"><li><span><a href="#XGBoost-Tuning" data-toc-modified-id="XGBoost-Tuning-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span><a href="https://www.analyticsvidhya.com/blog/2016/03/complete-guide-parameter-tuning-xgboost-with-codes-python/" target="_blank">XGBoost Tuning</a></a></span><ul class="toc-item"><li><span><a href="#Objective-function" data-toc-modified-id="Objective-function-2.1.1"><span class="toc-item-num">2.1.1&nbsp;&nbsp;</span>Objective function</a></span></li><li><span><a href="#Tune-number-of-trees" data-toc-modified-id="Tune-number-of-trees-2.1.2"><span class="toc-item-num">2.1.2&nbsp;&nbsp;</span>Tune number of trees</a></span></li><li><span><a href="#Tune-tree-specific-parameters" data-toc-modified-id="Tune-tree-specific-parameters-2.1.3"><span class="toc-item-num">2.1.3&nbsp;&nbsp;</span>Tune tree-specific parameters</a></span><ul class="toc-item"><li><span><a href="#Tune-max_depth,-min_child_weight" data-toc-modified-id="Tune-max_depth,-min_child_weight-2.1.3.1"><span class="toc-item-num">2.1.3.1&nbsp;&nbsp;</span>Tune max_depth, min_child_weight</a></span></li><li><span><a href="#Tune-gamma" data-toc-modified-id="Tune-gamma-2.1.3.2"><span class="toc-item-num">2.1.3.2&nbsp;&nbsp;</span>Tune gamma</a></span></li><li><span><a href="#Tune-subsample,-colsample_bytree" data-toc-modified-id="Tune-subsample,-colsample_bytree-2.1.3.3"><span class="toc-item-num">2.1.3.3&nbsp;&nbsp;</span>Tune subsample, colsample_bytree</a></span></li></ul></li><li><span><a href="#Tune-regularization-parameters" data-toc-modified-id="Tune-regularization-parameters-2.1.4"><span class="toc-item-num">2.1.4&nbsp;&nbsp;</span>Tune regularization parameters</a></span></li><li><span><a href="#Lower-the-learning-rate-and-decide-the-optimal-parameters" data-toc-modified-id="Lower-the-learning-rate-and-decide-the-optimal-parameters-2.1.5"><span class="toc-item-num">2.1.5&nbsp;&nbsp;</span>Lower the learning rate and decide the optimal parameters</a></span></li></ul></li><li><span><a href="#LogisticRegression-Tuning" data-toc-modified-id="LogisticRegression-Tuning-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>LogisticRegression Tuning</a></span></li><li><span><a href="#Optional-MongoTrials" data-toc-modified-id="Optional-MongoTrials-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Optional <a href="https://hyperopt.github.io/hyperopt/scaleout/mongodb/" target="_blank">MongoTrials</a></a></span><ul class="toc-item"><li><span><a href="#XGBoost-Tuning" data-toc-modified-id="XGBoost-Tuning-2.3.1"><span class="toc-item-num">2.3.1&nbsp;&nbsp;</span>XGBoost Tuning</a></span></li></ul></li></ul></li><li><span><a href="#Results" data-toc-modified-id="Results-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Results</a></span></li></ul></div>

Продолжаем работать над задачей CTR-prediction с использованием датасета от Criteo.

Описание задачи и данных можно посмотреть в notebook'e предыдущей практики (`sgd_logreg_nn/notebooks/ctr_prediction_mllib.ipynb`).

In [1]:
! pip3.5 install xgboost

     |████████████████████████████████| 109.7 MB 11.2 MB/s eta 0:00:01   |█████████████████               | 58.3 MB 9.2 MB/s eta 0:00:06     |████████████████████▍           | 69.8 MB 11.7 MB/s eta 0:00:04     |████████████████████████▍       | 83.6 MB 12.7 MB/s eta 0:00:03     |█████████████████████████       | 85.4 MB 12.7 MB/s eta 0:00:02     |██████████████████████████▊     | 91.5 MB 12.3 MB/s eta 0:00:02


In [2]:
%matplotlib inline
%config InlineBackend.figure_format ='retina'

import os
import sys
import glob
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

import pyspark
import pyspark.sql.functions as F
from pyspark.conf import SparkConf
from pyspark.sql import SQLContext
from pyspark.sql import SparkSession
from pyspark.sql import Row

COMMON_PATH = '/workspace/common'

sys.path.append(os.path.join(COMMON_PATH, 'utils'))

os.environ['PYSPARK_SUBMIT_ARGS'] = """
--jars {common}/xgboost4j-spark-0.72.jar,{common}/xgboost4j-0.72.jar
--py-files {common}/sparkxgb.zip pyspark-shell
""".format(common=COMMON_PATH).replace('\n', ' ')

spark = SparkSession \
    .builder \
    .master('local[*]') \
    .appName("spark_sql_examples") \
    .config("spark.executor.memory", "12g") \
    .config("spark.driver.memory", "12g") \
    .config("spark.task.cpus", "8") \
    .config("spark.executor.cores", "8") \
    .getOrCreate()

sc = spark.sparkContext
sqlContext = SQLContext(sc)

from metrics import rocauc, logloss, ne, get_ate
from processing import split_by_col

from sparkxgb.xgboost import *

In [3]:
DATA_PATH = '/workspace/data/criteo'

TRAIN_PATH = os.path.join(DATA_PATH, 'train.csv')

In [4]:
df = sqlContext.read.format("com.databricks.spark.csv") \
    .option("delimiter", ",") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load('file:///' + TRAIN_PATH)

**Remark** Необязательно использовать половину датасета и всего две категориальные переменные. Можно использовать больше данных, если вам позволяет ваша конфигурация

In [5]:
df = df.sample(False, 0.5)

In [6]:
num_columns = ['_c{}'.format(i) for i in range(1, 14)]
cat_columns = ['_c{}'.format(i) for i in range(14, 40)][:2]
len(num_columns), len(cat_columns)

(13, 2)

In [7]:
df = df.fillna(0, subset=num_columns)

In [8]:
from pyspark.ml import PipelineModel


pipeline_model = PipelineModel.load(os.path.join(DATA_PATH, 'pipeline_model'))

In [9]:
df = pipeline_model \
    .transform(df) \
    .select(F.col('_c0').alias('label'), 'features', 'id') \
    .cache()

df.count()

1831732

In [10]:
train_df, val_df, test_df = split_by_col(df, 'id', [0.8, 0.1, 0.1])

# [Spark MLlib Tuning](https://spark.apache.org/docs/latest/ml-tuning.html)

У имеющегося в Spark'e метода HPO есть два существенных недостатка, которые делают его мало пригодным в контексте нашей задачи:

1. `ParamGridBuilder` - поиск по сетке
2. `TrainValidationSplit` - делит данные случайнм образом

# [Hyperopt](https://github.com/hyperopt/hyperopt)

Установим `hyperopt`

In [11]:
!pip3.5 install hyperopt

## [XGBoost Tuning](https://www.analyticsvidhya.com/blog/2016/03/complete-guide-parameter-tuning-xgboost-with-codes-python/)

> [Notes on Parameter Tuning](https://xgboost.readthedocs.io/en/latest/tutorials/param_tuning.html)

### Objective function

In [12]:
from hyperopt import fmin, tpe, hp, Trials, STATUS_OK
import scipy.stats as st


def objective(space):
    estimator = XGBoostEstimator(**space)
    print('SPACE:', estimator._input_kwargs_processed())
    success = False
    attempts = 0
    model = None
    while not success and attempts < 2:
        try:
            model = estimator.fit(train_df)
            success = True
        except Exception as e:
            attempts += 1
            print(e)
            print('Try again')
        
    log_loss = logloss(model, val_df, probabilities_col='probabilities')
    roc_auc = rocauc(model, val_df, probabilities_col='probabilities')
    
    print('LOG-LOSS: {}, ROC-AUC: {}'.format(log_loss, roc_auc))

    return {'loss': log_loss, 'rocauc': roc_auc, 'status': STATUS_OK }

In [13]:
static_params = {
    'featuresCol': "features", 
    'labelCol': "label", 
    'predictionCol': "prediction",
    'eval_metric': 'logloss',
    'objective': 'binary:logistic',
    'nthread': 8,
    'silent': 0,
    'nworkers': 1
}

Fix baseline parameters and train baseline model

In [14]:
%%time
CONTROL_NAME = 'xgb baseline'

baseline_params = {
    'colsample_bytree': 0.9,
    'eta': 0.15,
    'gamma': 0.9,
    'max_depth': 6,
    'min_child_weight': 50.0,
    'subsample': 0.9,
    'num_round': 20
}

baseline_model = XGBoostEstimator(**{**static_params, **baseline_params}).fit(train_df)

CPU times: user 50.9 ms, sys: 22 ms, total: 73 ms
Wall time: 13.3 s


In [15]:
baseline_rocauc = rocauc(baseline_model, val_df, probabilities_col='probabilities')
baseline_rocauc

0.726643973947602

In [16]:
all_metrics = {}

In [17]:
baseline_test_metrics = {
    'logloss': logloss(baseline_model, test_df, probabilities_col='probabilities'),
    'rocauc': rocauc(baseline_model, test_df, probabilities_col='probabilities')
}

all_metrics[CONTROL_NAME] = baseline_test_metrics

### Tune number of trees

> Choose a relatively high learning rate. Generally a learning rate of 0.1 works but somewhere between 0.05 to 0.3 should work for different problems. Determine the optimum number of trees for this learning rate.

In [18]:
%%time

num_round_choice = [10, 20, 40, 100]
eta_choice = [0.5, 0.10, 0.15, 0.20, 0.30]

space = {
    # Optimize
    'num_round': hp.choice('num_round', num_round_choice),
    'eta': hp.choice('eta', eta_choice),
    
    # Fixed    
    'max_depth': baseline_params['max_depth'],
    'min_child_weight': baseline_params['min_child_weight'],
    'subsample': baseline_params['subsample'],
    'gamma': baseline_params['gamma'],
    'colsample_bytree': baseline_params['colsample_bytree'],
    
    **static_params
}


trials = Trials()
best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=20,
            trials=trials)

SPACE:                                                
{'eta': 0.2, 'predictionCol': 'prediction', 'min_child_weight': 50.0, 'eval_metric': 'logloss', 'colsample_bytree': 0.9, 'nthread': 8, 'labelCol': 'label', 'gamma': 0.9, 'nworkers': 1, 'silent': 0, 'featuresCol': 'features', 'max_depth': 6, 'num_round': 20, 'objective': 'binary:logistic', 'subsample': 0.9}
LOG-LOSS: 0.5070732747524984, ROC-AUC: 0.728915913948984
SPACE:                                                                          
{'eta': 0.15, 'predictionCol': 'prediction', 'min_child_weight': 50.0, 'eval_metric': 'logloss', 'colsample_bytree': 0.9, 'nthread': 8, 'labelCol': 'label', 'gamma': 0.9, 'nworkers': 1, 'silent': 0, 'featuresCol': 'features', 'max_depth': 6, 'num_round': 40, 'objective': 'binary:logistic', 'subsample': 0.9}
LOG-LOSS: 0.5048485205140673, ROC-AUC: 0.7319716131842697                       
SPACE:                                                                          
{'eta': 0.5, 'predictionCol'

LOG-LOSS: 0.5006837529610447, ROC-AUC: 0.7376339682524755                         
SPACE:                                                                            
{'eta': 0.15, 'predictionCol': 'prediction', 'min_child_weight': 50.0, 'eval_metric': 'logloss', 'colsample_bytree': 0.9, 'nthread': 8, 'labelCol': 'label', 'gamma': 0.9, 'nworkers': 1, 'silent': 0, 'featuresCol': 'features', 'max_depth': 6, 'num_round': 100, 'objective': 'binary:logistic', 'subsample': 0.9}
LOG-LOSS: 0.5016334748200031, ROC-AUC: 0.7363495981148913                         
SPACE:                                                                            
{'eta': 0.2, 'predictionCol': 'prediction', 'min_child_weight': 50.0, 'eval_metric': 'logloss', 'colsample_bytree': 0.9, 'nthread': 8, 'labelCol': 'label', 'gamma': 0.9, 'nworkers': 1, 'silent': 0, 'featuresCol': 'features', 'max_depth': 6, 'num_round': 40, 'objective': 'binary:logistic', 'subsample': 0.9}
LOG-LOSS: 0.5037516134696022, ROC-AUC: 0.733482274

In [19]:
best

{'eta': 0, 'num_round': 3}

Обратите внимание на то, что в случае с `hp.choice` в переменной `best` хранится не конкретное значение гиперпараметра, а его индекс из списка, например, `num_round_choice`

In [20]:
eta = eta_choice[best['eta']]  # change me!
num_round = num_round_choice[best['num_round']]  # change me!

### Tune tree-specific parameters

> Tune tree-specific parameters ( max_depth, min_child_weight, gamma, subsample, colsample_bytree) for decided learning rate and number of trees. Note that we can choose different parameters to define a tree and I’ll take up an example here.

#### Tune max_depth, min_child_weight

In [35]:
%%time

max_depth_choice  = list(range(3, 9))
min_child_weight_choice = list(map(float, range(0, 100, 15)))

space = {
    # Optimize
    'max_depth': hp.choice('max_depth', max_depth_choice),
    'min_child_weight': hp.choice('min_child_weight', min_child_weight_choice),
    
    # Fixed
    'num_round': num_round,
    'eta': eta,
    'subsample': baseline_params['subsample'],
    'gamma': baseline_params['gamma'],
    'colsample_bytree': baseline_params['colsample_bytree'],
    
    **static_params
}


trials = Trials()
best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=20,
            trials=trials)

SPACE:                                                
{'eta': 0.5, 'predictionCol': 'prediction', 'min_child_weight': 45.0, 'eval_metric': 'logloss', 'colsample_bytree': 0.9, 'nthread': 8, 'labelCol': 'label', 'gamma': 0.9, 'nworkers': 1, 'silent': 0, 'featuresCol': 'features', 'max_depth': 4, 'num_round': 100, 'objective': 'binary:logistic', 'subsample': 0.9}
LOG-LOSS: 0.5005462632689146, ROC-AUC: 0.7376582232785444
SPACE:                                                                          
{'eta': 0.5, 'predictionCol': 'prediction', 'min_child_weight': 0.0, 'eval_metric': 'logloss', 'colsample_bytree': 0.9, 'nthread': 8, 'labelCol': 'label', 'gamma': 0.9, 'nworkers': 1, 'silent': 0, 'featuresCol': 'features', 'max_depth': 7, 'num_round': 100, 'objective': 'binary:logistic', 'subsample': 0.9}
LOG-LOSS: 0.4999247301586791, ROC-AUC: 0.7382783750532562                       
SPACE:                                                                          
{'eta': 0.5, 'predictionCol

LOG-LOSS: 0.4989976084213681, ROC-AUC: 0.7395239117870991                        
SPACE:                                                                           
{'eta': 0.5, 'predictionCol': 'prediction', 'min_child_weight': 45.0, 'eval_metric': 'logloss', 'colsample_bytree': 0.9, 'nthread': 8, 'labelCol': 'label', 'gamma': 0.9, 'nworkers': 1, 'silent': 0, 'featuresCol': 'features', 'max_depth': 3, 'num_round': 100, 'objective': 'binary:logistic', 'subsample': 0.9}
LOG-LOSS: 0.5024359718981231, ROC-AUC: 0.7349200303592547                        
SPACE:                                                                           
{'eta': 0.5, 'predictionCol': 'prediction', 'min_child_weight': 30.0, 'eval_metric': 'logloss', 'colsample_bytree': 0.9, 'nthread': 8, 'labelCol': 'label', 'gamma': 0.9, 'nworkers': 1, 'silent': 0, 'featuresCol': 'features', 'max_depth': 3, 'num_round': 100, 'objective': 'binary:logistic', 'subsample': 0.9}
LOG-LOSS: 0.5026264658719934, ROC-AUC: 0.7347071626637

In [36]:
best

{'max_depth': 3, 'min_child_weight': 5}

In [37]:
best_max_depth = max_depth_choice[best['max_depth']]
best_min_child_weight = min_child_weight_choice[best['min_child_weight']]
best_max_depth, best_min_child_weight

(6, 75.0)

#### Tune gamma

In [43]:
%%time

gamma_choice = np.logspace(-3, 0, num=10).tolist()

space = {
    # Optimize
    'gamma': hp.choice('gamma', gamma_choice),

    # Fixed
    'num_round': num_round,
    'eta': eta,
    'max_depth': best_max_depth,
    'min_child_weight': best_min_child_weight,
    'subsample': baseline_params['subsample'],
    'colsample_bytree': baseline_params['colsample_bytree'],
    
    **static_params
}


trials = Trials()
best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=10,
            trials=trials)

SPACE:                                                
{'eta': 0.5, 'predictionCol': 'prediction', 'min_child_weight': 75.0, 'eval_metric': 'logloss', 'colsample_bytree': 0.9, 'nthread': 8, 'labelCol': 'label', 'gamma': 1.0, 'nworkers': 1, 'silent': 0, 'featuresCol': 'features', 'max_depth': 6, 'num_round': 100, 'objective': 'binary:logistic', 'subsample': 0.9}
LOG-LOSS: 0.4992144004826749, ROC-AUC: 0.7393219041988388
SPACE:                                                                          
{'eta': 0.5, 'predictionCol': 'prediction', 'min_child_weight': 75.0, 'eval_metric': 'logloss', 'colsample_bytree': 0.9, 'nthread': 8, 'labelCol': 'label', 'gamma': 0.21544346900318823, 'nworkers': 1, 'silent': 0, 'featuresCol': 'features', 'max_depth': 6, 'num_round': 100, 'objective': 'binary:logistic', 'subsample': 0.9}
LOG-LOSS: 0.49891763957303975, ROC-AUC: 0.7396034139770696                      
SPACE:                                                                           
{'eta': 0

In [44]:
best

{'gamma': 8}

In [46]:
best_gamma = gamma_choice[best['gamma']]
best_gamma

0.46415888336127775

#### Tune subsample, colsample_bytree

In [53]:
%%time

subsample_choice = np.linspace(0.3, 1, num=10).tolist()
colsample_bytree_choice = np.linspace(0.3, 1, num=10).tolist()

space = {
    # Optimize
    'subsample': hp.choice('subsample', subsample_choice),
    'colsample_bytree': hp.choice('colsample_bytree', colsample_bytree_choice),
    
    # Fixed
    'num_round': num_round,
    'eta': eta,
    'max_depth': best_max_depth,
    'min_child_weight': best_min_child_weight,
    'gamma': best_gamma,
    
    **static_params
}


trials = Trials()
best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=20,
            trials=trials)

SPACE:                                                
{'eta': 0.5, 'predictionCol': 'prediction', 'min_child_weight': 75.0, 'eval_metric': 'logloss', 'colsample_bytree': 0.8444444444444446, 'nthread': 8, 'labelCol': 'label', 'gamma': 0.46415888336127775, 'nworkers': 1, 'silent': 0, 'featuresCol': 'features', 'max_depth': 6, 'num_round': 100, 'objective': 'binary:logistic', 'subsample': 0.37777777777777777}
LOG-LOSS: 0.5008803212776852, ROC-AUC: 0.7369094563135963
SPACE:                                                                          
{'eta': 0.5, 'predictionCol': 'prediction', 'min_child_weight': 75.0, 'eval_metric': 'logloss', 'colsample_bytree': 0.9222222222222223, 'nthread': 8, 'labelCol': 'label', 'gamma': 0.46415888336127775, 'nworkers': 1, 'silent': 0, 'featuresCol': 'features', 'max_depth': 6, 'num_round': 100, 'objective': 'binary:logistic', 'subsample': 0.45555555555555555}
LOG-LOSS: 0.500113178576608, ROC-AUC: 0.7380824243355362                        
SPACE:       

SPACE:                                                                            
{'eta': 0.5, 'predictionCol': 'prediction', 'min_child_weight': 75.0, 'eval_metric': 'logloss', 'colsample_bytree': 0.5333333333333333, 'nthread': 8, 'labelCol': 'label', 'gamma': 0.46415888336127775, 'nworkers': 1, 'silent': 0, 'featuresCol': 'features', 'max_depth': 6, 'num_round': 100, 'objective': 'binary:logistic', 'subsample': 0.3}
LOG-LOSS: 0.5017695565304562, ROC-AUC: 0.7358670153064175                        
SPACE:                                                                           
{'eta': 0.5, 'predictionCol': 'prediction', 'min_child_weight': 75.0, 'eval_metric': 'logloss', 'colsample_bytree': 0.8444444444444446, 'nthread': 8, 'labelCol': 'label', 'gamma': 0.46415888336127775, 'nworkers': 1, 'silent': 0, 'featuresCol': 'features', 'max_depth': 6, 'num_round': 100, 'objective': 'binary:logistic', 'subsample': 0.9222222222222223}
LOG-LOSS: 0.49899612344749655, ROC-AUC: 0.7395240110534231

In [54]:
best

{'colsample_bytree': 7, 'subsample': 7}

In [55]:
best_colsample_bytree = colsample_bytree_choice[best['colsample_bytree']]
best_subsample = subsample_choice[best['subsample']]
best_colsample_bytree, best_subsample

(0.8444444444444446, 0.8444444444444446)

### Tune regularization parameters

> Tune regularization parameters (lambda, alpha) for xgboost which can help reduce model complexity and enhance performance.

In [70]:
%%time

lambda_choice = [0.] + np.logspace(-5, 0, num=6).tolist()
alpha_choice = [0.] + np.logspace(-5, 0, num=6).tolist()

space = {
    # Optimize
    'reg_lambda': hp.choice('reg_lambda', lambda_choice),
    'alpha': hp.choice('alpha', alpha_choice),
    
    # Fixed
    'num_round': num_round,
    'eta': eta,
    'max_depth': best_max_depth,
    'min_child_weight': best_min_child_weight,
    'gamma': best_gamma,
    'subsample': best_subsample,
    'colsample_bytree': best_colsample_bytree,
    
    **static_params
}


trials = Trials()
best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=20,
            trials=trials)
best

SPACE:                                                
{'colsample_bytree': 0.8444444444444446, 'lambda': 1.0, 'eta': 0.5, 'predictionCol': 'prediction', 'min_child_weight': 75.0, 'eval_metric': 'logloss', 'max_depth': 6, 'nthread': 8, 'alpha': 0.1, 'subsample': 0.8444444444444446, 'gamma': 0.46415888336127775, 'silent': 0, 'labelCol': 'label', 'num_round': 100, 'objective': 'binary:logistic', 'nworkers': 1, 'featuresCol': 'features'}
LOG-LOSS: 0.49894927116659543, ROC-AUC: 0.7394596453917597
SPACE:                                                                           
{'colsample_bytree': 0.8444444444444446, 'lambda': 1e-05, 'eta': 0.5, 'predictionCol': 'prediction', 'min_child_weight': 75.0, 'eval_metric': 'logloss', 'max_depth': 6, 'nthread': 8, 'alpha': 0.0001, 'subsample': 0.8444444444444446, 'gamma': 0.46415888336127775, 'silent': 0, 'labelCol': 'label', 'num_round': 100, 'objective': 'binary:logistic', 'nworkers': 1, 'featuresCol': 'features'}
LOG-LOSS: 0.49920824690329707, 

LOG-LOSS: 0.49920824690329707, ROC-AUC: 0.7392289967223893                        
SPACE:                                                                            
{'colsample_bytree': 0.8444444444444446, 'lambda': 0.001, 'eta': 0.5, 'predictionCol': 'prediction', 'min_child_weight': 75.0, 'eval_metric': 'logloss', 'max_depth': 6, 'nthread': 8, 'alpha': 0.01, 'subsample': 0.8444444444444446, 'gamma': 0.46415888336127775, 'silent': 0, 'labelCol': 'label', 'num_round': 100, 'objective': 'binary:logistic', 'nworkers': 1, 'featuresCol': 'features'}
LOG-LOSS: 0.4993197781680958, ROC-AUC: 0.7389702207892354                         
SPACE:                                                                            
{'colsample_bytree': 0.8444444444444446, 'lambda': 1e-05, 'eta': 0.5, 'predictionCol': 'prediction', 'min_child_weight': 75.0, 'eval_metric': 'logloss', 'max_depth': 6, 'nthread': 8, 'alpha': 0.001, 'subsample': 0.8444444444444446, 'gamma': 0.46415888336127775, 'silent': 0, 'label

{'alpha': 2, 'reg_lambda': 6}

In [71]:
best

{'alpha': 2, 'reg_lambda': 6}

In [72]:
best_alpha = alpha_choice[best['alpha']]
best_lambda = lambda_choice[best['reg_lambda']]
best_alpha, best_lambda

(0.0001, 1.0)

### Lower the learning rate and decide the optimal parameters

In [ ]:
######################################
######### YOUR CODE HERE #############
######################################

---
## LogisticRegression Tuning

Подберем гиперпараметры для логрега из предыдущих практик

In [ ]:
######################################
######### YOUR CODE HERE #############
######################################

---
## Optional [MongoTrials](https://hyperopt.github.io/hyperopt/scaleout/mongodb/)

> For parallel search, hyperopt includes a MongoTrials implementation that supports asynchronous updates.

**TLDR** Преимущества использования `MongoTrials`:
* `MongoTrials` позволяет параллельно запускать несколько вычислений целевой функции
* Динамический уровень параллелизма - можно добавлять/удалять воркеров, которые вычисляют целевую функцию
* Все результаты сохраняются в БД - история запусков никуда не потеряется

*За выполнение данного задания можно получить дополнительно +0.4 к итоговому баллу*

### XGBoost Tuning

In [ ]:
######################################
######### YOUR CODE HERE #############
######################################

# Results

Подведем итоги.

Обучите модели с найденными (оптимальными) гиперпараметрами и сделайте справнение на отложенной выборке

#### XGB

In [75]:
best_params = {
    # Fixed
    'num_round': num_round,
    'eta': eta,
    'max_depth': best_max_depth,
    'min_child_weight': best_min_child_weight,
    'gamma': best_gamma,
    'subsample': best_subsample,
    'colsample_bytree': best_colsample_bytree,
    'reg_lambda': best_lambda,
    'alpha': best_alpha
}

In [76]:
tuned_model = XGBoostEstimator(**{**static_params, **best_params}).fit(train_df)

In [77]:
tuned_xgb_test_metrics = {
    'logloss': logloss(tuned_model, test_df, probabilities_col='probabilities'),
    'rocauc': rocauc(tuned_model, test_df, probabilities_col='probabilities')
}

all_metrics['tuned_xgb'] = tuned_xgb_test_metrics

Итоговая таблица

In [78]:
get_ate(all_metrics, CONTROL_NAME)

,metric,tuned_xgb ate %
0,rocauc,1.892389
1,logloss,-2.145114


выглядит неплохо